In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
import numpy as np
import pandas as pd
import scipy as sp
import scipy.ndimage
from rdkit import Chem 
from rdkit.Chem import rdqueries , rdCoordGen, Lipinski , AllChem, Draw
from rdkit.Chem.rdmolfiles import SDWriter
import MDAnalysis as mda
from MDAnalysis import Universe, Merge, transformations
from scipy.spatial import ConvexHull
import scipy.integrate as integrate

# Calculating Volumes

In [2]:
colnames=["AA","RefractiveIndex", "Charge", "SMILES","Mass","Volume","Volume_corr", "Radii","Radii_corr",'Density' ] 
aminoAcidData = pd.read_csv(r'data/aminoAcidData_2023_js.csv', names=colnames, header = None, index_col = 0)

In [3]:
#atom RBondi (Å) 
#Yuan H. Zhao, Michael H. Abraham,* and Andreas M. Zissimos†
#J. Org. Chem, Vol. 68, No. 19, 2003 7369
bondiDic={
    "H" : 1.20 ,
    "P" : 1.80 ,
    "C" : 1.70 ,
    "S" : 1.80 ,
    "N" : 1.55 ,
    "As" : 1.85 ,
    "O" : 1.52 ,
    "B" : 2.13 ,
    "F" : 1.47 ,
    "Si" : 2.10 ,
    "Cl" : 1.75 ,
    "Se" : 1.90 ,
    "Br" : 1.85 ,
    "Te" : 2.06 ,
    "I" : 1.98      
}

In [4]:
aaOnly={"GLY":"GLY","ALA":"ALA","ARG":"ARG","ASN":"ASN",
      "ASP":"ASP","CYS":"CYS","GLN":"GLN","GLU":"GLU",
      "HIS":"HIS","ILE" :"ILE","LEU":"LEU","LYS":"LYS",
      "MET":"MET","PHE" :"PHE","PRO":"PRO","SER":"SER", 
      "THR":"THR","TRP" :"TRP","TYR":"TYR","VAL":"VAL",
      "HSD":"HID","HSE" :"HIE","HSP":"HIP", "HIS":"HIS",
      "CYM":"CYM","ASPP":"PAS","GLUP":"PGL"}

In [5]:
#notes:
#name of bead's pdb = name of CHARMM residue (to find later the starting point)
#name of UA bead should consist of 3 letters otherwise UA can not see it
#clean PDBs after running canonical rotations & propka to have 3-letter UA residue names
#ensure that UA residues in PDB has al least one CA atom name (check for non-aminoacid residues specifically), re-name where necessarily 
charmm_to_UA3Letter={"GLY":"GLY","ALA":"ALA","ARG":"ARG","ASN":"ASN",
                      "ASP":"ASP","CYS":"CYS","GLN":"GLN","GLU":"GLU",
                      "HIS":"HIS","ILE" :"ILE","LEU":"LEU","LYS":"LYS",
                      "MET":"MET","PHE" :"PHE","PRO":"PRO","SER":"SER", 
                      "THR":"THR","TRP" :"TRP","TYR":"TYR","VAL":"VAL",
                      "HSD":"HID","HSE" :"HIE","HSP":"HIP", "HIS":"HIS",
                      "CYM":"CYM","PASP":"PAS","PGLU":"PGL",
                      "BGLC":"BGL","AMAN":"MAN","AFUC":"FUC", #carbs
                      "BGLCNA":"NGL","BGALNA":"NGA", #N-glycolization
                      "NC4":"NC4","MAS":"MAS","DMEP":"DMP","MAMM":"MMM", #lipids
                      "BTE2":"BTE", "CHOL":"CHO", "CHL1":"CHL", #lipids
                      "PROA":"PRA"} #extras: propionic acid                 

In [6]:
wd=os.getcwd()
beadsDir=wd+"/beads/"  #folder with crd and psf files, can be pdb as well

In [7]:
def fill_with_points(density, origin, rShpere):
    phi         = np.random.uniform( 0.0, 2.0*np.pi, size=(density,))
    theta_cos   = np.random.uniform(-1.0,       1.0, size=(density,))
    u           = np.random.uniform( 0.0,       1.0, size=(density,))
    theta_sin   = np.sqrt(1.0 - theta_cos**2.0)
    r           = rShpere * np.cbrt(u)
    return np.array([
        np.array([
            origin[0] + r[i] * theta_sin[i] * np.cos(phi[i]),
            origin[1] + r[i] * theta_sin[i] * np.sin(phi[i]),
            origin[2] + r[i] * theta_cos[i]
        ]) for i in range(density)
    ])

In [8]:
sdfOut=wd+"/CustomAABeadSet.sdf"
writer = SDWriter(sdfOut)
crds=[]
for crd in os.listdir(beadsDir):
    if crd.endswith(".crd"):
                crds.append(crd)
for CRD in crds:
    crdIn=beadsDir+CRD
    psfIn=beadsDir+CRD.rsplit(".")[0]+".psf"
    AA=charmm_to_UA3Letter[CRD.rsplit(".")[0].upper()]
    uIn = Universe(psfIn,crdIn)
    guessed_elements = mda.topology.guessers.guess_types(uIn.atoms.names)
    uIn.add_TopologyAttr('elements', guessed_elements)
    mass=np.sum(uIn.atoms.masses)
    mol = uIn.atoms.convert_to("RDKIT")
    mol.SetProp('ID', "CHARMM RESI name:" +CRD.rsplit(".")[0])
    mol.SetProp('_Name', AA)
    rdCoordGen.AddCoords(mol)
    writer.write(mol)
    smile=Chem.MolToSmiles(mol)
    shapeArr=[]
    for atom in uIn.atoms:
        array=(fill_with_points(20000, atom.position, bondiDic[atom.element]))
        for i, coor in enumerate(array):
            shapeArr.append([coor[0],coor[1], coor[2]])
    a3_to_nm3 = 10 ** (-3)
    a3_to_cm3 = 10 ** (-24)
    vol_cm3    = a3_to_cm3*ConvexHull(shapeArr).volume # in nm^3/g
    vol        = a3_to_nm3*ConvexHull(shapeArr).volume # in nm^3/g
    radius     = ((3/(4*sp.constants.pi))*vol) ** (1. / 3.) # radii
    density    = mass/(vol_cm3*sp.constants.Avogadro) # for density plot [g/cm3]
    vol_corr   = vol*(density/1.5) # 1.5 is desired density
    radius_corr = ((3/(4*sp.constants.pi))*vol_corr) ** (1. / 3.) # radii   
    aminoAcidData['SMILES'][AA]    = smile
    aminoAcidData['Mass'][AA]      = round(mass, 1)   #in g/mol
    aminoAcidData['Density'][AA]   = round(density, 3) #in g/cm3
    aminoAcidData['Volume'][AA]    = round(vol ,3)   #in nm*3
    aminoAcidData['Radii'][AA]     = round(radius, 3) #in nm
    aminoAcidData['Volume_corr'][AA]    = round(vol_corr , 3)   #in nm*3
    aminoAcidData['Radii_corr'][AA]     = round(radius_corr, 3) #in nm

<h1>Print strings for config file and list of beads.</h1>

In [9]:
print("amino-acids = [ " + ", ".join([ "{:.6}".format(r) for r in aminoAcidData.index ]) + " ]")
print("amino-acid-charges = [ " + ", ".join([ "{:.3f}".format(r) for r in aminoAcidData['Charge'].values ]) + " ]")
print("amino-acid-radii = [ " + ", ".join([ "{:.3f}".format(r) for r in aminoAcidData['Radii_corr'].values ]) + " ]")


amino-acids = [ GLY, ALA, ARG, ASN, ASP, CYS, GLN, GLU, HID, HIE, HIP, HIS, ILE, LEU, LYS, MET, PHE, PRO, SER, THR, TRP, TYR, VAL, CYM, PAS, PGL, BGL, MAN, FUC, NGL, NGA, NC4, MAS, DMP, MMM, BTE, CHO, CHL ]
amino-acid-charges = [ 0.000, 0.000, 1.000, 0.000, -1.000, 0.000, 0.000, -1.000, 0.000, 0.000, 1.000, 0.500, 0.000, 0.000, 1.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, -1.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 0.000, 1.000, 0.000, -1.000, 1.000, 0.000, 1.000, 0.000 ]
amino-acid-radii = [ 0.271, 0.266, 0.346, 0.311, 0.311, 0.301, 0.324, 0.324, 0.331, 0.331, 0.332, 0.332, 0.310, 0.310, 0.324, 0.326, 0.339, 0.295, 0.284, 0.299, 0.366, 0.351, 0.297, 0.317, 0.328, 0.339, 0.362, 0.362, 0.351, 0.388, 0.388, 0.270, 0.270, 0.321, 0.204, 0.246, 0.302, 0.468 ]


In [10]:
#List of beads to include in UA autogenerated config files, providing the three-letter code, charge and radius to write to config files.
#Used in corona tool
#BeadID,Charge[e],Radius[nm]
f = open("CustomAABeadSet.csv", "a")
f.write("#BeadID,Charge[e],Radius[nm]\n")
for i,aa in enumerate(aminoAcidData.index.tolist()):
    chrg=aminoAcidData['Charge'].values.tolist()[i]
    rds=aminoAcidData['Radii_corr'].values.tolist()[i]
    f.write(("{},{},{}\n").format(aa, chrg,rds))
f.close()

<h1> Hamaker Constant Calculation </h1>

In [8]:
kT            = 1.38064852e-23 * 300
h             = 6.62607e-34

In [9]:
class Material:
    def __init__(self, name, e, n , v):
        self.name = name
        self._e    = e  # Dielectric Constant
        self._n    = n  # Refractive Index
        self._v    = v  # Peak adsorption frequency

In [10]:
def HamakerInsulator(material, aminoAcid):
    global kT
    global h
    global aminoAcidData
    
    n1 = aminoAcidData['RefractiveIndex'][aminoAcid]  # Amino Acid 
    n2 = material._n                                  # Nanomaterial
    n3 = 1.33                                         # Water

    e1 = n1 ** 2.0    # Amino Acid
    e2 = material._e  # Nanomaterial
    e3 = 81.0         # Water

    ve = material._v # Peak adsorption for the material in the UV
    
    first  = (3.0 / 4.0) * kT * ((e1 - e3) / (e1 + e3)) * ((e2 - e3) / (e2 + e3))
    coef   = ((3.0 * h * ve) / (8.0 * np.sqrt(2.0)))
    npart  = ((n1 * n1 - n3 * n3) * (n2 * n2 - n3 * n3)) / (np.sqrt((n1 * n1 + n3 * n3) * (n2 * n2 + n3 * n3)) * (np.sqrt(n1 * n1 + n3 * n3) + np.sqrt(n2 * n2 + n3 * n3)))
    second = coef * npart
    
    #print(first, second)
    
    return first + second

In [11]:
def HamakerMetal(material, aminoAcid):
    global kT
    global h
    global refractiveIndex

    n1 = aminoAcidData['RefractiveIndex'][aminoAcid]  # Amino Acid    
    n2 = material._n                                  # Nanomaterial
    n3 = 1.32                                         # Water

    v1 = 2.5e15      # Amino Acid (Rough estimate based on other organic molecules in Israelachvili book)    
    v2 = material._v # Nanomaterial
    v3 = 3.0e15      # Water (from Israelachvili book)

    
    coef  = (3.0 * h ) / (8.0 * np.sqrt(2.0))
    npart = (n1 * n1 - n3 * n3) / (n1 * n1 + n3 * n3)
    vpart = (v2 * np.sqrt(v1 * v3)) / (np.sqrt(v1 * v3) + v2 / np.sqrt(n1 * n1 - n3 * n3))
      
    return coef * npart * vpart

In [13]:
def HamakerMetalIntegrand(v, n1, n3, v1, v2, v3):
    v1Sq = v1**2
    v2Sq = v2**2
    v3Sq = v3**2
    n1SqP1 = n1**2 + 1
    n3SqP1 = n3**2 + 1
    vSq = v**2
    return ((v2Sq/vSq - (n3SqP1*v3Sq)/(v3Sq + vSq))*((n1SqP1*v1Sq)/(v1Sq + vSq) - (n3SqP1*v3Sq)/(v3Sq + vSq)))/  ((2 + v2Sq/vSq + (n3SqP1*v3Sq)/(v3Sq + vSq))*(2 + (n1SqP1*v1Sq)/(v1Sq + vSq) + (n3SqP1*v3Sq)/(v3Sq + vSq)))

def HamakerMetalScaledIntegrand(x, n1, n3, v1, v2, v3):
    n3Sq = n3**2
    xSq = x**2
    alpha32 = (v2**2) /(v3**2)
    alpha31 = (v1**2) / (v3**2)
    beta31 = (n1**2) / (n3**2 )
    return ((alpha32 + xSq + alpha32*xSq - n3Sq*xSq)*(-(alpha31*xSq) - (-1 + n3Sq)*xSq + 
       alpha31*n3Sq*(-1 + beta31 + beta31*xSq)))/  ((alpha32*(1 + xSq) + xSq*(1 + n3Sq + 2*xSq))*
     (xSq*(1 + n3Sq + 2*xSq) + alpha31*(xSq + n3Sq*(1 + beta31 + beta31*xSq))))

def HamakerMetalFull(material, aminoAcidN):
    '''Calculate the Hamaker constant for dielectric - water - metal using an improved expression'''
    global kT
    global h
    global refractiveIndex
    n1 = aminoAcidN #aminoAcidData['RefractiveIndex'][aminoAcid]  # Amino Acid    
    n2 = material._n                                  # Nanomaterial
    n3 = 1.32                                         # Water

    v1 = 2.5e15      # Amino Acid (Rough estimate based on other organic molecules in Israelachvili book)    
    v2 = material._v # Nanomaterial
    v3 = 3.0e15      # Water (from Israelachvili book)

    e1 = n1**2.0  #CG bead
    e2 = 500000 #infinity for metals, unused
    e3 = 81.0 #water
    
    zeroFreqTerm = 3.0/4.0 * kT * (e1-e3)/(e1+e3)
    minFreq = (2*np.pi * kT/h)
    hamakerIntegrand = lambda v: HamakerMetalIntegrand(v, n1, n3, v1, v2, v3)
    intRes = integrate.quad( hamakerIntegrand, minFreq, 30e15)
    #print(intRes)
    hamakerIntegrandScaled = lambda v: HamakerMetalScaledIntegrand(v, n1, n3, v1, v2, v3)
    intRes = integrate.quad(hamakerIntegrandScaled, minFreq/v3 , 60e15/v3)
    #print(intRes)
    hamakerIntegrated = v3 * intRes[0]
    hamakerFreqTerm = 3 * h/(np.pi * 4) * hamakerIntegrated
    #print(zeroFreqTerm, hamakerFreqTerm)
    return zeroFreqTerm + hamakerFreqTerm
    
for aminoAcidN in np.arange(1.0, 5.1, 0.1):
    testCase = HamakerMetalFull(Material('Al', None,  1.37289, 3e16/ (2 * np.pi))   , aminoAcidN  )
    print(aminoAcidN, testCase/kT)

1.0 -24.954405731367554
1.1 -17.820127359332275
1.2000000000000002 -10.841515285411873
1.3000000000000003 -4.076676541691447
1.4000000000000004 2.435078797186484
1.5000000000000004 8.66946499623147
1.6000000000000005 14.613829891075042
1.7000000000000006 20.264267973821852
1.8000000000000007 25.623256567618075
1.9000000000000008 30.697777284167884
2.000000000000001 35.49785879349937
2.100000000000001 40.03546966145021
2.200000000000001 44.32369312785883
2.300000000000001 48.376123712690536
2.4000000000000012 52.206435177129116
2.5000000000000013 55.82807885436248
2.6000000000000014 59.25407985379167
2.7000000000000015 62.49690582754939
2.8000000000000016 65.5683888551279
2.9000000000000017 68.47968567513941
3.0000000000000018 71.24126514969954
3.100000000000002 73.86291466992398
3.200000000000002 76.35375936823054
3.300000000000002 78.72228963807356
3.400000000000002 80.9763936923833
3.500000000000002 83.12339281276016
3.6000000000000023 85.17007762654747
3.7000000000000024 87.12274425

In [14]:
#
# NM  parameters
#

gold            = Material('Au', None,  0.18104, 3.00e16 / (2 * np.pi)) # wavelength, as per VL suggestion;  Sopra Material Library https://www.filmetrics.com/refractive-index-database/Au/Gold
silver          = Material('Ag', None, 0.13511, 3.00e16/ (2 * np.pi)) #  wavelength, as per VL suggestion; P. Winsemius, F. F. van Kampen, H. P. Lengkeek, and C. G. van Went, J. Phys. F 6, 1583 (1976) 
copper          = Material('Cu', None,  0.23883, 3.00e16/ (2 * np.pi)) #  wavelength, as per VL suggestion; H. J. Hagemann, W. Gudat, and C. Kunz, DESY Report SR-74/7, Hamburg, 1974; H. J. Hagemann, W. Gudat, and C. Kunz, J. Opt. Soc. Am. 65, 742 (1975) 
nickel          = Material('Ni', None,  1.95776, 3.00e16/ (2 * np.pi) ) #  wavelength, as per VL suggestion; H. J. Hagemann, W. Gudat, and C. Kunz, DESY Report SR-74/7, Hamburg, 1974; H.
iron            = Material('Fe', None, 2.9304, 5.10e14)
aluminium         = Material('Al', None,  1.37289, 3.23e15) # renamed from alumina - this is metallic Al, not Al oxide

peg             = Material('PEG', 17.3, 1.493, 9.99e14)
ps              = Material('PS', 2.500, 1.711, 1.15e15)
cooh             = Material('COOH', 33.5, 1.375, 1.00e15)
coo              = Material('COO', 33.5, 1.375, 1.00e15)
nh3              = Material('NH3', 33.5, 1.000, 1.00e15)
nh2              = Material('NH2', 33.5, 1.000, 1.00e15)
cit              = Material('CIT', 33.5, 1.500, 1.153e15)

sio2_quartz     = Material('SiO2_Quartz', 4.27,  1.5459, 3.13e15)
sio2_amorphious = Material('SiO2_Amorphous', 3.75,  1.4599, 3.20e15)
ti02_rutile     = Material('TiO2_Rutile', 160,  2.6479, 1.24e15)
ti02_anatase    = Material('TiO2_Anatase', 160,  2.5287, 1.50e15)
cdse            = Material('CdSe', 20,  2.5516, 1.42e15)


semiconductors = [ sio2_quartz, sio2_amorphious, ti02_rutile, ti02_anatase, cdse ]
metals = [ gold, silver, copper, nickel, iron,  aluminium]
organics = [ peg, ps, cit, cooh, coo, nh3, nh2 ]


for material in organics:
    handle = open(material.name + '.dat', 'w')
    handle.write("#{:<6}{:<10}{:<14}{:<10}\n".format("Name", "kT", "Joules", "kJ/mol"))
    for aminoAcid in aminoAcidData.index.values:
        hamakerConstant = HamakerInsulator(material, aminoAcid)
        handle.write("{:<7}{:<10.3f}{:<14.3E}{:<10.3f}\n".format(aminoAcid, hamakerConstant / kT, hamakerConstant, hamakerConstant / kT / 0.4))
    handle.close()

for material in metals:
    handle = open(material.name + '.dat', 'w')
    handle.write("#{:<6}{:<10}{:<14}{:<10}\n".format("Name", "kT", "Joules", "kJ/mol"))
    for aminoAcid in aminoAcidData.index.values:
        hamakerConstant = HamakerMetalFull(material, aminoAcidData['RefractiveIndex'][aminoAcid])
        handle.write("{:<7}{:<10.3f}{:<14.3E}{:<10.3f}\n".format(aminoAcid, hamakerConstant / kT, hamakerConstant, hamakerConstant / kT / 0.4))
    handle.close()
    
for material in metals:
    handle = open(material.name + '_approx.dat', 'w')
    handle.write("#{:<6}{:<10}{:<14}{:<10}\n".format("Name", "kT", "Joules", "kJ/mol"))
    for aminoAcid in aminoAcidData.index.values:
        hamakerConstant = HamakerMetal(material, aminoAcid)
        handle.write("{:<7}{:<10.3f}{:<14.3E}{:<10.3f}\n".format(aminoAcid, hamakerConstant / kT, hamakerConstant, hamakerConstant / kT / 0.4))
    handle.close()
    
for material in semiconductors:
    handle = open(material.name + '.dat', 'w')
    handle.write("#{:<6}{:<10}{:<14}{:<10}\n".format("Name", "kT", "Joules", "kJ/mol"))
    for aminoAcid in aminoAcidData.index.values:
        hamakerConstant = HamakerInsulator(material, aminoAcid)
        handle.write("{:<7}{:<10.3f}{:<14.3E}{:<10.3f}\n".format(aminoAcid, hamakerConstant / kT, hamakerConstant, hamakerConstant / kT / 0.4))
    handle.close()